In [1]:
# работа с данными
import pandas as pd
import numpy as np

# графики
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

# настройка графиков
# matplotlib.rcParams['figure.figsize'] = (8, 8)
# sns.set_style('whitegrid')

# предупреждения
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

# tqdm
from tqdm.notebook import tqdm
# time
import itertools
# copy
from copy import deepcopy

# работа с файлами
import pickle
import json

In [2]:
#unlabeled = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/unlabeled_data.csv")
train  = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/train.csv")
test = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/test.csv")

In [3]:
train.head()

,id,sentiment,review
0,c77111d1e3526ac0f7cf13af46b65e65,1,I thought the film could be a bit more complex...
1,c4806c50f6560aa65f15ed69a25940f2,0,...except for Jon Heder. This guy tanked the e...
2,82b113c68594c74ab8dbe630206ec28a,1,"Jim Carrey is good as usual, and even though t..."
3,3d9891905de05248dfc2550239bd4687,1,Just read the original story which is written ...
4,da8d5a23f01e129b534022149825caa2,1,I watched this film over a hundred times. It i...


In [4]:
train['review'][0]

"I thought the film could be a bit more complex,in a psychological sense perhaps, but the action and voice acting were top notch. The animation was heavy CG in many scenes, but very good ones at that. This is one of the Batman Returns/Forever type films, which include romances and the conflicts of Wayne and motives for dating. 007 fans would love this, and so would the females, great theme song! Wayne was portrayed very well in this film, and the Penquin was back to his true form, no mutant genes in him this time! I liked the fact Robin wasn't used too much, Tim Drake was just a good computer nerd, somewhat of an Indigo child or mind of the future.<br /><br />The supporting cast was made up of some soap opera stars, decent talents and the characters were drawn to look like the voice actors too. Kelly Ripa was hilarious in this film.<br /><br />I rate this below Phantasm, Return of the Joker, and Batman vs. Dracula, but liked the smarter script better than I enjoyed Subzero. 7/10"

### Vader

Valence Aware Dictionary for sEntiment Reasoning

his lexical dictionary does not only contain words, but also phrases (such as “bad ass” and “the bomb”), emoticons (such as “:-)”) and sentiment-laden acronyms (such as “ROFL” and “WTF”). All the lexical features were rated for the polarity and intensity on a scale from “-4: Extremely Negative” to “+4 Extremely Positive” by 10 independent human raters. The average score is then used as the sentiment indicator for each lexical feature in the dictionary. For example, in Vader, the word “okay” has a positive rating of 0.9, “good” is 1.9 and “great” is 3.1, whereas “horrible” is -2.5, the frowning emoticon “:(“ is -2.2, and “sucks” is -1.5. Vader’s lexicon dictionary contains around 7,500 sentiment features in total and any word not listed in the dictionary will be scored as “0: Neutral”.

In [7]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ranas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
sia.polarity_scores('I am so happy!')

{'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.6468}

In [11]:
sia.polarity_scores(train['review'][0])

{'neg': 0.039, 'neu': 0.663, 'pos': 0.298, 'compound': 0.9963}

In [13]:
sia.polarity_scores(train['review'][1])

{'neg': 0.188, 'neu': 0.689, 'pos': 0.123, 'compound': -0.8989}

In [14]:
sia.polarity_scores(train['review'][2])

{'neg': 0.04, 'neu': 0.799, 'pos': 0.161, 'compound': 0.7904}

In [15]:
sia.polarity_scores(train['review'][3])

{'neg': 0.021, 'neu': 0.728, 'pos': 0.251, 'compound': 0.9437}

In [16]:
sia.polarity_scores(train['review'][4])

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.9694}

А оно неплохо справляется!

In [17]:
res = {}
for i, row in tqdm(train.iterrows(), total=len(train)):
    text = row['review']
    myid = row['id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/12500 [00:00<?, ?it/s]

In [19]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
vaders = vaders.merge(train, how='left')

In [20]:
vaders.head()

,id,neg,neu,pos,compound,sentiment,review
0,c77111d1e3526ac0f7cf13af46b65e65,0.039,0.663,0.298,0.9963,1,I thought the film could be a bit more complex...
1,c4806c50f6560aa65f15ed69a25940f2,0.188,0.689,0.123,-0.8989,0,...except for Jon Heder. This guy tanked the e...
2,82b113c68594c74ab8dbe630206ec28a,0.040,0.799,0.161,0.7904,1,"Jim Carrey is good as usual, and even though t..."
3,3d9891905de05248dfc2550239bd4687,0.021,0.728,0.251,0.9437,1,Just read the original story which is written ...
4,da8d5a23f01e129b534022149825caa2,0.000,0.508,0.492,0.9694,1,I watched this film over a hundred times. It i...


In [21]:
vaders['predict'] = np.where(vaders['compound']>0,1,0)

In [25]:
from sklearn.metrics import accuracy_score

train_metric = accuracy_score(
    y_true=vaders['sentiment'],
    y_pred=vaders['predict']
)

print(f"Accuracy score: {round(train_metric, 4)}")

Accuracy score: 0.6956


### Roberta pretrained model

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [6]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [7]:
encoded_text = tokenizer(train['review'][0], return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.0050721676, 'roberta_neu': 0.047950763, 'roberta_pos': 0.9469772}


In [22]:
import sys
sys.path.insert(1, 'C:/Users/ranas/ML course/Competitions/Second competition/functions')

In [50]:
import importlib
importlib.reload(sys.modules['data_preprocess'])
from data_preprocess import light_text_cleaning

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ranas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
train = light_text_cleaning(train, 'review')
test = light_text_cleaning(test, 'review')

  0%|          | 0/12500 [00:00<?, ?it/s]

C:\Users/ranas/ML course/Competitions/Second competition/functions\data_preprocess.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ranas\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


  0%|          | 0/12500 [00:00<?, ?it/s]

C:\Users\ranas\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [53]:
train.head()

,id,sentiment,review,tidy_review,review_length
0,c77111d1e3526ac0f7cf13af46b65e65,1,I thought the film could be a bit more complex...,I thought the film could be a bit more complex...,992
1,c4806c50f6560aa65f15ed69a25940f2,0,...except for Jon Heder. This guy tanked the e...,except for Jon Heder This guy tanked the entir...,842
2,82b113c68594c74ab8dbe630206ec28a,1,"Jim Carrey is good as usual, and even though t...",Jim Carrey is good as usual and even though th...,494
3,3d9891905de05248dfc2550239bd4687,1,Just read the original story which is written ...,Just read the original story which is written ...,370
4,da8d5a23f01e129b534022149825caa2,1,I watched this film over a hundred times. It i...,I watched this film over a hundred times It is...,177


In [58]:
train['tidy_review_length'] = train['tidy_review'].apply(lambda x: len(x))
train['tidy_review_length'].describe()

count    12500.000000
mean      1249.223440
std        947.274424
min         51.000000
25%        668.000000
50%        922.000000
75%       1515.250000
max       9057.000000
Name: tidy_review_length, dtype: float64

In [59]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, max_length = 511, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [62]:
random_idxs = np.random.permutation(np.arange(len(train)))[:1000]

In [63]:
res = {}
for i, row in tqdm(train.iloc[random_idxs,:].iterrows(), total=1000):
    try:
        text = row['review']
        myid = row['id']
        roberta_result = polarity_scores_roberta(text)
        res[myid] = roberta_result
    except RuntimeError:
        print(f'Broke for id {myid}')
        res[myid] = {
        'roberta_neg' : 0.5,
        'roberta_neu' : 0.5,
        'roberta_pos' : 0.5
        }
        

  0%|          | 0/12500 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [64]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'id'})
results_df = results_df.merge(train, how='inner')

In [65]:
results_df

,id,roberta_neg,roberta_neu,roberta_pos,sentiment,review,tidy_review,review_length,tidy_review_length
0,ebbb95a49a8ef8569e4d4599c98987bb,0.826881,0.145836,0.027282,0,I was going to say this was the worst gay-them...,I was going to say this was the worst gaytheme...,1023,961
1,ee2510a8636106af3f9a8e91e73aba84,0.227832,0.355267,0.416901,1,Most people who have seen this movie thinks th...,Most people who have seen this movie thinks th...,462,453
2,662331eaa956cc8da3acf61c4f442ec0,0.131450,0.343830,0.524720,1,This film starts out with a family who were al...,This film starts out with a family who were al...,918,900
3,46b9f84d8d96b41b537f0dd227adad2f,0.360152,0.488255,0.151593,1,I've often wondered just how much CASPER was m...,Ive often wondered just how much CASPER was me...,1067,989
4,8633392ec58c652aa65d4a4fd0695532,0.201731,0.383159,0.415110,1,"In my years of attending film festivals, I hav...",In my years of attending film festivals I have...,1059,991
...,...,...,...,...,...,...,...,...,...
995,e644e92dc717f32240de4e832f7d64f4,0.768460,0.173570,0.057970,0,"Since \Rugrats\""' falling from the category of...",Since Rugrats falling from the category of goo...,1212,1136
996,27bb1a8fd1178dd1800db4612c872d1f,0.002819,0.017365,0.979817,1,This movie was great! It was an excellent rend...,This movie was great It was an excellent rendi...,615,578
997,efd3b96e67c6871731f7ccd934b6c393,0.532550,0.416470,0.050980,0,"This is not the video nastie, but only because...",This is not the video nastie but only because ...,1385,1300
998,0143cb662c15f59aed603eafb570729e,0.596073,0.304863,0.099064,1,"I used to write comments at IMDb, but I don't ...",I used to write comments at IMDb but I dont do...,1179,1134


In [66]:
from sklearn.metrics import roc_auc_score
roc_auc_score(results_df['sentiment'], results_df['roberta_pos'])

0.9359175917591759

Трансформер, обученный на отзывах imbd

In [72]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [73]:
from transformers import pipeline
sentiment_model = pipeline(model="lvwerra/distilbert-imdb")

Downloading:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [76]:
sentiment_model(results_df['tidy_review'][3])

[{'label': 'POSITIVE', 'score': 0.631574809551239}]

In [79]:
MODEL = f"lvwerra/distilbert-imdb"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [81]:
encoded_text = tokenizer(results_df['tidy_review'][3], return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'imdb_neg' : scores[0],
    'imdb_pos' : scores[1]
}
print(scores_dict)

{'imdb_neg': 0.3684252, 'imdb_pos': 0.63157475}


In [82]:
def polarity_scores_imdb(example):
    encoded_text = tokenizer(example, max_length = 511, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'imdb_neg' : scores[0],
        'imdb_pos' : scores[1]
    }
    return scores_dict

In [83]:
res_imdb = {}
for i, row in tqdm(results_df.iterrows(), total=len(results_df)):
    try:
        text = row['tidy_review']
        myid = row['id']
        imdb_result = polarity_scores_imdb(text)
        res_imdb[myid] = imdb_result
    except RuntimeError:
        print(f'Broke for id {myid}')
        res_imdb[myid] = {
        'imdb_neg' : scores[0],
        'imdb_pos' : scores[1]
        }

  0%|          | 0/1000 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [95]:
imdb_results_df = pd.DataFrame(res_imdb).T
imdb_results_df = imdb_results_df.reset_index().rename(columns={'index': 'id'})
imdb_results_df = imdb_results_df.merge(results_df, how='inner')

In [97]:
imdb_results_df.head()

,id,imdb_neg,imdb_pos,roberta_neg,roberta_neu,roberta_pos,sentiment,review,tidy_review,review_length,tidy_review_length
0,ebbb95a49a8ef8569e4d4599c98987bb,0.996249,0.003751,0.826881,0.145836,0.027282,0,I was going to say this was the worst gay-them...,I was going to say this was the worst gaytheme...,1023,961
1,ee2510a8636106af3f9a8e91e73aba84,0.016645,0.983355,0.227832,0.355267,0.416901,1,Most people who have seen this movie thinks th...,Most people who have seen this movie thinks th...,462,453
2,662331eaa956cc8da3acf61c4f442ec0,0.009330,0.990670,0.131450,0.343830,0.524720,1,This film starts out with a family who were al...,This film starts out with a family who were al...,918,900
3,46b9f84d8d96b41b537f0dd227adad2f,0.368425,0.631575,0.360152,0.488255,0.151593,1,I've often wondered just how much CASPER was m...,Ive often wondered just how much CASPER was me...,1067,989
4,8633392ec58c652aa65d4a4fd0695532,0.136969,0.863031,0.201731,0.383159,0.415110,1,"In my years of attending film festivals, I hav...",In my years of attending film festivals I have...,1059,991


In [98]:
from sklearn.metrics import roc_auc_score
roc_auc_score(imdb_results_df['sentiment'], imdb_results_df['imdb_pos'])

0.9852305230523052